In [739]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Importing Data

In [740]:
order = pd.read_csv("_DA_Order_Brushing/order_brush_order.csv") #order data file
customer = pd.read_csv("_DA_Marketing_Analytics/train.csv") #main user data file
customer_age = pd.read_csv("_DA_Marketing_Analytics/users.csv") #age domain etc
products = pd.read_csv("products.csv")


In [741]:
customer = pd.merge(customer, customer_age, how='left', on='user_id')

In [742]:
customer.country_code.value_counts()

country_code
1    28423
2    13874
3    13845
4     6302
5     5536
6     3221
7     2338
Name: count, dtype: int64

In [743]:
SG_customer = customer[customer.country_code == 7].reset_index()
print(SG_customer)

      index  country_code                 grass_date  user_id  \
0      2393             7  2019-07-17 00:00:00+08:00      299   
1      2429             7  2019-07-17 00:00:00+08:00     2143   
2      2434             7  2019-07-17 00:00:00+08:00     2306   
3      2448             7  2019-07-17 00:00:00+08:00     3334   
4      2486             7  2019-07-17 00:00:00+08:00     5751   
...     ...           ...                        ...      ...   
2333  69594             7  2019-09-01 00:00:00+08:00   126576   
2334  69599             7  2019-09-01 00:00:00+08:00   126948   
2335  69607             7  2019-09-01 00:00:00+08:00   127209   
2336  69611             7  2019-09-01 00:00:00+08:00   127441   
2337  69619             7  2019-09-01 00:00:00+08:00   127744   

      subject_line_length last_open_day last_login_day last_checkout_day  \
0                      32            19              1                 2   
1                      32            21              2             

In [744]:
order

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [745]:
products

,price_ori,delivery,item_category_detail,specification,title,w_date,link_ori,item_rating,seller_name,idElastic,price_actual,sitename,idHash,total_rating,id,total_sold,pict_link,favorite,timestamp,desc
0,25.80,"KL City, Kuala Lumpur",Shopee | Women's Bags | Shoulder Bags | Should...,Product Specifications Category Shopee Women's...,1081 PU SLING BAG KULIT leather Casual Beg Tan...,2023-04-27,https://shopee.com.my/1081-PU-SLING-BAG-KULIT-...,4.9,qqfwholesale,19ec9fdedd9fe3e14bc652950fc3c423,12.90,shopee,88f051f0a2dcf0d29191e0175b828938,179,88f051f0a2dcf0d29191e0175b828938,179,https://down-my.img.susercontent.com/file/my-1...,Favorite (128,1682553600000,🎉 WELCOME To QQF WHOLESALE \r\n😍 ALL item read...
1,1.41,NaN,Shopee | Home Appliances | Sewing Machines | S...,Product Specifications Category Shopee Home Ap...,Stat Chinese Traditional Frog Buttons Knot Che...,2023-04-27,https://shopee.com.my/Stat-Chinese-Traditional...,5.0,stationery.my,5322309be3b6d9c01d15fa19f5b7ad9a,0.75,shopee,472d22f58af3bb35cf28c6eed7088dc5,17,472d22f58af3bb35cf28c6eed7088dc5,17,https://down-my.img.susercontent.com/file/4147...,Favorite (19,1682553600000,Chinese Traditional Frog Buttons Knot Cheongsa...
2,8.50,"KL City, Kuala Lumpur",Shopee | Women's Bags | Handbags | Set Handbags,Product Specifications Category Shopee Women's...,N09 READY STOCK MYFOOYIN woman handbag set 2in...,2023-04-26,https://shopee.com.my/N09-READY-STOCK-MYFOOYIN...,4.9,fooyin88,7a36765d1e2ca2b6a5414ddfeedfefc3,8.50,shopee,ec6ce64f4b77efb38b6392818959f181,8.1k,ec6ce64f4b77efb38b6392818959f181,8.1k,https://down-my.img.susercontent.com/file/23da...,Favorite (21.5k,1682467200000,READY STOCK\nMEASUREMENT:\nLONG 26CM X HEIGHT ...
3,8.70,NaN,Shopee | Women Clothes | Maternity Wear | Inne...,Product Specifications Category Shopee Women C...,🔥Shocking Sale🔥Cotton Comfort Nursing Bra Brea...,2023-04-26,https://shopee.com.my/%F0%9F%94%A5Shocking-Sal...,4.9,vivi2002,7fc4ab69722e4f788f217b4c044f83eb,8.70,shopee,ef4e0e17fdc7d0125d8609dcc395bc3d,6.5k,ef4e0e17fdc7d0125d8609dcc395bc3d,6.5k,https://down-my.img.susercontent.com/file/df47...,Favorite (3.4k,1682467200000,PRODUCT DESCRIPTION\nMaterial:95%Cotton+5%Span...
4,3999.00,"KL City, Kuala Lumpur",Shopee | Cameras & Drones | DSLR Cameras | DSLRs,Product Specifications Category Shopee Cameras...,Canon EOS 70D 18-55mm IS STM Original+64gb+ext...,2023-04-26,https://shopee.com.my/Canon-EOS-70D-18-55mm-IS...,5.0,xpro.gadget,55608f901f4c5384f884de18227d4298,2739.00,shopee,568c2029fed7d22973ff96d0404a8ff8,10,568c2029fed7d22973ff96d0404a8ff8,10,https://down-my.img.susercontent.com/file/4d3a...,Favorite (132,1682467200000,Canon 70D\nHighlights\nEF-S 18-55mm f/3.5-5.6 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20307,50.60,NaN,Shopee | Baby & Toys | Girls Fashion | Girl Shoes,Product Specifications Category Shopee Baby & ...,Elegant All-match Girls Leather Flat Shoes Pea...,2023-05-13,https://shopee.com.my/Elegant-All-match-Girls-...,4.9,amour.my,fe0f926e9b0588513b6d0e284c80a6b0,30.36,shopee,d8455ff553c9a2739d1bb6bd1f775f7c,219,d8455ff553c9a2739d1bb6bd1f775f7c,219,https://down-my.img.susercontent.com/file/1b02...,Favorite (1.6k,1683936000000,1 cm = 0.3937 inch \r\n-----------------------...
20308,32.00,NaN,Shopee | Baby & Toys | Girls Fashion | Girl Dr...,Product Specifications Category Shopee Baby & ...,Baju Raya kids 2023(v1) | Shopee Malaysia,2023-05-13,https://shopee.com.my/Baju-Raya-kids-2023%28v1...,4.8,mazu9,eeb4fbeff8aad77f6c316de399b30516,32.00,shopee,50bf7d6c1a1a2c812de434ab593f619d,215,50bf7d6c1a1a2c812de434ab593f619d,215,https://down-my.img.susercontent.com/file/my-1...,Favorite (304,1683936000000,M G FASHION\n-stylist fashionable\n-comfortab...
20309,4.98,NaN,Shopee | Muslim Fashion | Kain Pasang & Songke...,Product Specifications Category Shopee Muslim ...,Kain como crepe heavy premium (harga untk 0.5m...,2023-05-13,https://shopee.com.my/Kain-como-crepe-heavy-pr...,4.7,nurlainafashionzone,159cc3d6a46

# Data Cleaning for Customer

In [746]:
SG_customer.isna().sum()

index                             0
country_code                      0
grass_date                        0
user_id                           0
subject_line_length               0
last_open_day                     0
last_login_day                    0
last_checkout_day                 0
open_count_last_10_days           0
open_count_last_30_days           0
open_count_last_60_days           0
login_count_last_10_days          0
login_count_last_30_days          0
login_count_last_60_days          0
checkout_count_last_10_days       0
checkout_count_last_30_days       0
checkout_count_last_60_days       0
open_flag                         0
row_id                            0
attr_1                         1240
attr_2                            2
attr_3                            0
age                            1240
domain                            0
dtype: int64

In [747]:
SG_customer = SG_customer.drop(columns = ['index', 'country_code', 'grass_date'])
SG_customer = SG_customer.rename(columns={'user_id': 'customer_id'})

In [748]:
#filling out age NaNs based on Statista Records
# Define the age groups and corresponding probabilities

age_groups = [18, 25, 35, 45, 55]
age_probabilities = [0.13, 0.20, 0.22, 0.22, 0.23]

# Identify missing age values
missing_age_mask = SG_customer['age'].isna()

# Generate random ages according to the given probabilities for the missing values
random_ages = np.random.choice(age_groups, size=missing_age_mask.sum(), p=age_probabilities)

# Fill missing age values with the generated random ages
SG_customer.loc[missing_age_mask, 'age'] = random_ages

In [749]:
SG_customer

,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,...,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,attr_1,attr_2,attr_3,age,domain
0,299,32,19,1,2,0,1,3,34,118,...,1,2,7,0,2393,1.0,1.0,1.0,38.0,@gmail.com
1,2143,32,21,2,7,0,2,4,20,68,...,1,3,10,0,2429,1.0,1.0,2.0,26.0,@yahoo.com
2,2306,37,130,1,10,0,0,0,172,522,...,12,12,26,0,2434,1.0,1.0,4.0,56.0,@gmail.com
3,3334,32,9,13,19,1,1,2,2,10,...,0,1,3,0,2448,1.0,1.0,2.0,30.0,@gmail.com
4,5751,32,62,2,5,0,0,0,22,64,...,4,13,18,0,2486,1.0,1.0,2.0,27.0,@yahoo.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,126576,63,32,20,Never checkout,0,0,1,0,0,...,0,0,0,0,69594,NaN,1.0,0.0,25.0,@yahoo.com
2334,126948,63,5,5,Never checkout,1,3,3,0,0,...,0,0,0,0,69599,1.0,1.0,1.0,26.0,@gmail.com
2335,127209,63,85,889,Never checkout,0,0,0,0,0,...,0,0,0,0,69607,NaN,1.0,0.0,18.0,@yahoo.com
2336,127441,63,28,474,Never checkout,0,1,2,0,0,...,0,0,0,0,69611,1.0,1.0,1.0,30.0,@yahoo.com


# Data Cleaning For Products

In [750]:
# Use regular expressions to extract 'Stock' and 'Ships From' values
products['Stock'] = products['specification'].str.extract(r'Stock\s(\d+)').fillna(0).astype(int)
products['Ships_From'] = products['specification'].str.extract(r'Ships From\s(.+)$')

In [751]:
# MYR to SGD
products['price_ori'] = products['price_ori']/3.21118 
products['price_actual'] = products['price_actual']/3.21118 

In [752]:
def convert_rating(value):
    try:
        if 'k' in value:
            # Convert values like '8.1k' to 8100
            return int(float(value.replace('k', '')) * 1000)
        else:
            # Convert normal string numbers to integers
            return int(value)
    except:
        print(value)


# Apply the conversion function to the 'total_rating' column
products['total_rating'] = products['total_rating'].apply(convert_rating)
products['total_sold'] = products['total_sold'].apply(convert_rating)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [753]:
def convert_favorite(value):
    # Remove 'Favorite (' and convert the result to string
    try:
        value = value.replace('Favorite (', '').replace(')', '')
    except:
        print(value)
    return(convert_rating(value))

products['favorite'] = products['favorite'].apply(convert_favorite)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [754]:
products= products.drop(['delivery', 'specification', 'w_date', 'link_ori', 'idElastic', 'sitename', 'idHash', 'id', 'pict_link', 'timestamp'], axis=1)

In [755]:
products['product_id'] = products.index + 1

In [756]:
products

,price_ori,item_category_detail,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id
0,8.034430,Shopee | Women's Bags | Shoulder Bags | Should...,1081 PU SLING BAG KULIT leather Casual Beg Tan...,4.9,qqfwholesale,4.017215,179.0,179.0,128.0,🎉 WELCOME To QQF WHOLESALE \r\n😍 ALL item read...,2906,Johor,1
1,0.439091,Shopee | Home Appliances | Sewing Machines | S...,Stat Chinese Traditional Frog Buttons Knot Che...,5.0,stationery.my,0.233559,17.0,17.0,19.0,Chinese Traditional Frog Buttons Knot Cheongsa...,31701,Mainland China,2
2,2.647002,Shopee | Women's Bags | Handbags | Set Handbags,N09 READY STOCK MYFOOYIN woman handbag set 2in...,4.9,fooyin88,2.647002,8100.0,8100.0,21500.0,READY STOCK\nMEASUREMENT:\nLONG 26CM X HEIGHT ...,2044,Terengganu,3
3,2.709284,Shopee | Women Clothes | Maternity Wear | Inne...,🔥Shocking Sale🔥Cotton Comfort Nursing Bra Brea...,4.9,vivi2002,2.709284,6500.0,6500.0,3400.0,PRODUCT DESCRIPTION\nMaterial:95%Cotton+5%Span...,386,Pahang,4
4,1245.336605,Shopee | Cameras & Drones | DSLR Cameras | DSLRs,Canon EOS 70D 18-55mm IS STM Original+64gb+ext...,5.0,xpro.gadget,852.957480,10.0,10.0,132.0,Canon 70D\nHighlights\nEF-S 18-55mm f/3.5-5.6 ...,36,Kuala Lumpur,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20307,15.757447,Shopee | Baby & Toys | Girls Fashion | Girl Shoes,Elegant All-match Girls Leather Flat Shoes Pea...,4.9,amour.my,9.454468,219.0,219.0,1600.0,1 cm = 0.3937 inch \r\n-----------------------...,19076,Mainland China,20308
20308,9.965184,Shopee | Baby & Toys | Girls Fashion | Girl Dr...,Baju Raya kids 2023(v1) | Shopee Malaysia,4.8,mazu9,9.965184,215.0,215.0,304.0,M G FASHION\n-stylist fashionable\n-comfortab...,9406,W.P. Kuala Lumpur,20309
20309,1.550832,Shopee | Muslim Fashion | Kain Pasang & Songke...,Kain como crepe heavy premium (harga untk 0.5m...,4.7,nurlainafashionzone,0.775416,152.0,152.0,98.0,Harga utk 0.5m@setengah meter / HARGA BUKAN U...,446287,Kuala Lumpur,20310
20310,0.093424,Shopee | Baby & Toys | Others | Others,Doorgift Bag Birthday Kids Drawstring Organize...,4.9,pbs_empire,0.093424,16.0,16.0,27.0,Custom made Ready Stock in Malaysia\nUnicorn S...,3695,Selangor,20311


In [757]:
products.Ships_From.value_counts()

Ships_From
Selangor                     6287
Mainland China               2655
Kuala Lumpur                 2219
Johor                        1377
Perak                        1322
                             ... 
Larut, 34600 Perak              1
Pitas, 89130 Sabah              1
Kota Setar, 05150 Kedah         1
Dungun, 23000 Terengganu        1
Shah Alam, 40100 Selangor       1
Name: count, Length: 422, dtype: int64

In [758]:
location_to_country_non_malaysia = {
    "Mainland China": "China",
    "Indonesia": "Indonesia",
    "Korea": "South Korea",
    "Vietnam": "Vietnam",
    "Taiwan": "Taiwan",
    "Thailand": "Thailand",
    "KOTA JAKARTA SELATAN - KEBAYORAN BARU, DKI JAKARTA, ID": "Indonesia",
    "KAB. MOJOKERTO - MOJOSARI, JAWA TIMUR, ID": "Indonesia",
    "KOTA JAKARTA UTARA - KELAPA GADING, DKI JAKARTA, ID": "Indonesia",
    "KOTA JAKARTA PUSAT - TANAH ABANG, DKI JAKARTA, ID": "Indonesia",
    "KOTA JAKARTA BARAT - KEBON JERUK, DKI JAKARTA, ID": "Indonesia"
}


# Apply mapping to 'Ship_From' column
products['Ships_From'] = products['Ships_From'].apply(lambda x: location_to_country_non_malaysia.get(x, "Malaysia"))



In [759]:
products.item_category_detail.value_counts().to_csv('Categories.csv')

# Data Cleaning Order

In [760]:
order

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


# Making Synthetic Data

In [761]:
# Loop through each customer and add random noise to their checkout count

# Ensure that checkout_count_last_1_year is always greater than checkout_count_last_60_days
SG_customer['checkout_count_last_1_year'] = SG_customer.apply(
    lambda row: row['checkout_count_last_60_days'] + np.random.randint(0, (row['checkout_count_last_60_days']+3)*6),  # Multiplier ensures it's larger
    axis=1
)


In [762]:
# Function to apply the logic based on 'last_checkout_day'
def process_checkout(row):
    if row['last_checkout_day'] == 'Never checkout':
        # Generate random integer 0 or 1
        random_value = np.random.randint(0, 2)
        
        # If random_value is 0 and checkout_count_last_year > 0
        if random_value == 0 and row['checkout_count_last_1_year'] > 0:
            # Set last_checkout_day to 60 + random number between 0 and 300
            row['last_checkout_day'] = 60 + np.random.randint(0, 301)
        else:
            row['checkout_count_last_1_year'] = 0
    
    # Return the updated row
    return row

# Apply the function to the DataFrame
SG_customer = SG_customer.apply(process_checkout, axis=1)


In [763]:
SG_customer

,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,...,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,attr_1,attr_2,attr_3,age,domain,checkout_count_last_1_year
0,299,32,19,1,2,0,1,3,34,118,...,2,7,0,2393,1.0,1.0,1.0,38.0,@gmail.com,56
1,2143,32,21,2,7,0,2,4,20,68,...,3,10,0,2429,1.0,1.0,2.0,26.0,@yahoo.com,77
2,2306,37,130,1,10,0,0,0,172,522,...,12,26,0,2434,1.0,1.0,4.0,56.0,@gmail.com,123
3,3334,32,9,13,19,1,1,2,2,10,...,1,3,0,2448,1.0,1.0,2.0,30.0,@gmail.com,9
4,5751,32,62,2,5,0,0,0,22,64,...,13,18,0,2486,1.0,1.0,2.0,27.0,@yahoo.com,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,126576,63,32,20,Never checkout,0,0,1,0,0,...,0,0,0,69594,NaN,1.0,0.0,25.0,@yahoo.com,0
2334,126948,63,5,5,Never checkout,1,3,3,0,0,...,0,0,0,69599,1.0,1.0,1.0,26.0,@gmail.com,0
2335,127209,63,85,889,Never checkout,0,0,0,0,0,...,0,0,0,69607,NaN,1.0,0.0,18.0,@yahoo.com,0
2336,127441,63,28,474,270,0,1,2,0,0,...,0,0,0,69611,1.0,1.0,1.0,30.0,@yahoo.com,7


In [764]:
sum(SG_customer.checkout_count_last_1_year)

69353

In [765]:
from datetime import datetime, timedelta

# Function to generate random order times within a specific time range
def generate_order_times(num_orders, start_date, end_date):
    return [start_date + (end_date - start_date) * np.random.random() for _ in range(num_orders)]

# Function to generate orders for each customer
def generate_orders_for_customer(customer_row, product_ids):
    orders = []

    # Generate order times for the last 30 days
    total_orders_last_30_days = customer_row['checkout_count_last_30_days']
    if total_orders_last_30_days > 0:
        start_30_days = datetime(2019, 11, 1)
        end_30_days = datetime(2019, 12, 31)
        order_times_last_30_days = generate_order_times(total_orders_last_30_days, start_30_days, end_30_days)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_30_days])
    
    # Generate order times for the last 60 days (excluding last 30 days)
    total_orders_last_60_days = customer_row['checkout_count_last_60_days'] - total_orders_last_30_days
    if total_orders_last_60_days > 0:
        start_60_days = datetime(2019, 10, 1)
        end_60_days = datetime(2019, 10, 31)
        order_times_last_60_days = generate_order_times(total_orders_last_60_days, start_60_days, end_60_days)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_60_days])
    
    # Generate order times for the rest of the year (excluding last 60 days)
    total_orders_last_1_year = customer_row['checkout_count_last_1_year'] - customer_row['checkout_count_last_60_days']
    if total_orders_last_1_year > 0:
        start_1_year = datetime(2019, 1, 1)
        end_1_year = datetime(2019, 9, 30)
        order_times_last_1_year = generate_order_times(total_orders_last_1_year, start_1_year, end_1_year)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_1_year])

    # Generate order IDs and construct order records
    order_ids = [f"order_{i}" for i in range(len(order_dataset) + 1, len(order_dataset) + len(orders) + 1)]
    
    orders_df = pd.DataFrame({
        'order_id': order_ids,
        'order_time': [order[0] for order in orders],
        'product_id': [order[1] for order in orders],
        'customer_id': [order[2] for order in orders]
    })

    return orders_df

# Initialize the order dataset
order_dataset = pd.DataFrame(columns=['order_id', 'order_time', 'product_id', 'customer_id'])

# Get product IDs from the product dataset
product_ids = products['product_id'].values

# Generate orders for each customer
for idx, customer_row in SG_customer.iterrows():
    customer_orders = generate_orders_for_customer(customer_row, product_ids)
    order_dataset = pd.concat([order_dataset, customer_orders], ignore_index=True)


C:\Users\MuhammadFauzan\AppData\Local\Temp\ipykernel_13756\48896343.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  order_dataset = pd.concat([order_dataset, customer_orders], ignore_index=True)


In [766]:
order_dataset

,order_id,order_time,product_id,customer_id
0,order_1,2019-11-28 23:57:44.878827,13340,299
1,order_2,2019-11-09 07:37:57.759814,1377,299
2,order_3,2019-10-02 16:07:15.863415,6999,299
3,order_4,2019-10-12 07:38:58.963653,8340,299
4,order_5,2019-10-14 15:26:09.099943,11729,299
...,...,...,...,...
69377,order_69378,2019-02-15 02:05:21.159518,1006,127744
69378,order_69379,2019-06-01 16:55:39.083033,5850,127744
69379,order_69380,2019-04-08 09:19:15.865389,13012,127744
69380,order_69381,2019-09-29 05:37:43.973087,8922,127744


In [767]:
products.columns

Index(['price_ori', 'item_category_detail', 'title', 'item_rating',
       'seller_name', 'price_actual', 'total_rating', 'total_sold', 'favorite',
       'desc', 'Stock', 'Ships_From', 'product_id'],
      dtype='object')

In [768]:
customer.columns

Index(['country_code', 'grass_date', 'user_id', 'subject_line_length',
       'last_open_day', 'last_login_day', 'last_checkout_day',
       'open_count_last_10_days', 'open_count_last_30_days',
       'open_count_last_60_days', 'login_count_last_10_days',
       'login_count_last_30_days', 'login_count_last_60_days',
       'checkout_count_last_10_days', 'checkout_count_last_30_days',
       'checkout_count_last_60_days', 'open_flag', 'row_id', 'attr_1',
       'attr_2', 'attr_3', 'age', 'domain'],
      dtype='object')

In [769]:
products

,price_ori,item_category_detail,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id
0,8.034430,Shopee | Women's Bags | Shoulder Bags | Should...,1081 PU SLING BAG KULIT leather Casual Beg Tan...,4.9,qqfwholesale,4.017215,179.0,179.0,128.0,🎉 WELCOME To QQF WHOLESALE \r\n😍 ALL item read...,2906,Malaysia,1
1,0.439091,Shopee | Home Appliances | Sewing Machines | S...,Stat Chinese Traditional Frog Buttons Knot Che...,5.0,stationery.my,0.233559,17.0,17.0,19.0,Chinese Traditional Frog Buttons Knot Cheongsa...,31701,China,2
2,2.647002,Shopee | Women's Bags | Handbags | Set Handbags,N09 READY STOCK MYFOOYIN woman handbag set 2in...,4.9,fooyin88,2.647002,8100.0,8100.0,21500.0,READY STOCK\nMEASUREMENT:\nLONG 26CM X HEIGHT ...,2044,Malaysia,3
3,2.709284,Shopee | Women Clothes | Maternity Wear | Inne...,🔥Shocking Sale🔥Cotton Comfort Nursing Bra Brea...,4.9,vivi2002,2.709284,6500.0,6500.0,3400.0,PRODUCT DESCRIPTION\nMaterial:95%Cotton+5%Span...,386,Malaysia,4
4,1245.336605,Shopee | Cameras & Drones | DSLR Cameras | DSLRs,Canon EOS 70D 18-55mm IS STM Original+64gb+ext...,5.0,xpro.gadget,852.957480,10.0,10.0,132.0,Canon 70D\nHighlights\nEF-S 18-55mm f/3.5-5.6 ...,36,Malaysia,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20307,15.757447,Shopee | Baby & Toys | Girls Fashion | Girl Shoes,Elegant All-match Girls Leather Flat Shoes Pea...,4.9,amour.my,9.454468,219.0,219.0,1600.0,1 cm = 0.3937 inch \r\n-----------------------...,19076,China,20308
20308,9.965184,Shopee | Baby & Toys | Girls Fashion | Girl Dr...,Baju Raya kids 2023(v1) | Shopee Malaysia,4.8,mazu9,9.965184,215.0,215.0,304.0,M G FASHION\n-stylist fashionable\n-comfortab...,9406,Malaysia,20309
20309,1.550832,Shopee | Muslim Fashion | Kain Pasang & Songke...,Kain como crepe heavy premium (harga untk 0.5m...,4.7,nurlainafashionzone,0.775416,152.0,152.0,98.0,Harga utk 0.5m@setengah meter / HARGA BUKAN U...,446287,Malaysia,20310
20310,0.093424,Shopee | Baby & Toys | Others | Others,Doorgift Bag Birthday Kids Drawstring Organize...,4.9,pbs_empire,0.093424,16.0,16.0,27.0,Custom made Ready Stock in Malaysia\nUnicorn S...,3695,Malaysia,20311


In [770]:
products['item_category_detail'].apply(lambda x: len(x.split('|'))).value_counts()

item_category_detail
4    19913
3      399
Name: count, dtype: int64

In [771]:
# Function to extract the categories by splitting by '|'
def extract_category_1(value):
    return value.split('|')[1].strip() if len(value.split('|')) > 1 else None
def extract_category_2(value):
    return value.split('|')[2].strip() if len(value.split('|')) > 2 else None
def extract_category_3(value):
    return value.split('|')[3].strip() if len(value.split('|')) > 3 else None

# Apply the function to the item_category_detail column
products['category_1'] = products['item_category_detail'].apply(extract_category_1)
products['category_2'] = products['item_category_detail'].apply(extract_category_2)
products['category_3'] = products['item_category_detail'].apply(extract_category_3)



In [772]:
products.drop(columns=['item_category_detail'], inplace=True)

In [773]:
products

,price_ori,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id,category_1,category_2,category_3
0,8.034430,1081 PU SLING BAG KULIT leather Casual Beg Tan...,4.9,qqfwholesale,4.017215,179.0,179.0,128.0,🎉 WELCOME To QQF WHOLESALE \r\n😍 ALL item read...,2906,Malaysia,1,Women's Bags,Shoulder Bags,Shoulder Bags
1,0.439091,Stat Chinese Traditional Frog Buttons Knot Che...,5.0,stationery.my,0.233559,17.0,17.0,19.0,Chinese Traditional Frog Buttons Knot Cheongsa...,31701,China,2,Home Appliances,Sewing Machines,Sewing Machines and Accessories
2,2.647002,N09 READY STOCK MYFOOYIN woman handbag set 2in...,4.9,fooyin88,2.647002,8100.0,8100.0,21500.0,READY STOCK\nMEASUREMENT:\nLONG 26CM X HEIGHT ...,2044,Malaysia,3,Women's Bags,Handbags,Set Handbags
3,2.709284,🔥Shocking Sale🔥Cotton Comfort Nursing Bra Brea...,4.9,vivi2002,2.709284,6500.0,6500.0,3400.0,PRODUCT DESCRIPTION\nMaterial:95%Cotton+5%Span...,386,Malaysia,4,Women Clothes,Maternity Wear,Innerwear
4,1245.336605,Canon EOS 70D 18-55mm IS STM Original+64gb+ext...,5.0,xpro.gadget,852.957480,10.0,10.0,132.0,Canon 70D\nHighlights\nEF-S 18-55mm f/3.5-5.6 ...,36,Malaysia,5,Cameras & Drones,DSLR Cameras,DSLRs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20307,15.757447,Elegant All-match Girls Leather Flat Shoes Pea...,4.9,amour.my,9.454468,219.0,219.0,1600.0,1 cm = 0.3937 inch \r\n-----------------------...,19076,China,20308,Baby & Toys,Girls Fashion,Girl Shoes
20308,9.965184,Baju Raya kids 2023(v1) | Shopee Malaysia,4.8,mazu9,9.965184,215.0,215.0,304.0,M G FASHION\n-stylist fashionable\n-comfortab...,9406,Malaysia,20309,Baby & Toys,Girls Fashion,Girl Dresses
20309,1.550832,Kain como crepe heavy premium (harga untk 0.5m...,4.7,nurlainafashionzone,0.775416,152.0,152.0,98.0,Harga utk 0.5m@setengah meter / HARGA BUKAN U...,446287,Malaysia,20310,Muslim Fashion,Kain Pasang & Songket,Kain Pasang Cotton
20310,0.093424,Doorgift Bag Birthday Kids Drawstring Organize...,4.9,pbs_empire,0.093424,16.0,16.0,27.0,Custom made Ready Stock in Malaysia\nUnicorn S...,3695,Malaysia,20311,Baby & Toys,Others,Others


In [774]:
print(products.category_1.value_counts(normalize=True)*100)

category_1
Health & Beauty           12.130760
Men Clothes               12.101221
Women Clothes             10.988578
Mobile & Accessories       9.290075
Baby & Toys                7.365104
Home & Living              6.478929
Groceries & Pets           5.563214
Home Appliances            4.667192
Automotive                 4.046869
Sports & Outdoor           3.879480
Women's Bags               3.067152
Watches                    2.624065
Men's Bags & Wallets       2.422213
Cameras & Drones           2.225286
Women Shoes                2.185900
Muslim Fashion             2.062820
Fashion Accessories        2.018511
Computer & Accessories     1.924970
Games, Books & Hobbies     1.698503
Men Shoes                  1.028948
Travel & Luggage           0.758173
Gaming & Consoles          0.728633
Tickets & Vouchers         0.521859
Others                     0.221544
Name: proportion, dtype: float64


In [775]:
# Rename 'event_time' to 'order_date' and convert it to datetime format
order.rename(columns={'event_time': 'order_date'}, inplace=True)
order['order_date'] = pd.to_datetime(order['order_date'], format='%Y-%m-%d %H:%M:%S')
order

,orderid,shopid,userid,order_date
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [776]:
# # Get product_id distribution by category from products df
# product_category_counts = products['category_1'].value_counts(normalize=True)

# # Create a list of product_ids weighted by their category distribution
# weighted_product_ids = []

# for category, proportion in product_category_counts.items():
#     # Get the product_ids belonging to the category
#     category_product_ids = products[products['category_1'] == category]['product_id']
    
#     # Randomly sample product_ids according to the proportion
#     category_sample_size = int(len(order) * proportion)
    
#     weighted_product_ids.extend(
#         np.random.choice(category_product_ids, size=category_sample_size, replace=True)
#     )

# # Shuffle the weighted product ids to ensure randomness
# np.random.shuffle(weighted_product_ids)

# # Ensure the number of product_ids matches the length of the order_df
# weighted_product_ids = np.random.choice(weighted_product_ids, size=len(order), replace=True)

# Create the delivery dataframe
delivery = order_dataset.copy()
# delivery['product_id'] = weighted_product_ids

# Map product_id back to its category
product_id_to_category = products.set_index('product_id')['category_1'].to_dict()

# Add category column to the delivery dataframe
delivery['category_1'] = delivery['product_id'].map(product_id_to_category)

# Display the distribution of the categories in the delivery dataframe
print(delivery['category_1'].value_counts(normalize=True)*100)


category_1
Health & Beauty           12.248134
Men Clothes               12.089591
Women Clothes             11.024473
Mobile & Accessories       9.271857
Baby & Toys                7.243954
Home & Living              6.644375
Groceries & Pets           5.714739
Home Appliances            4.594852
Automotive                 3.999596
Sports & Outdoor           3.783402
Women's Bags               3.121847
Watches                    2.662074
Men's Bags & Wallets       2.418495
Cameras & Drones           2.288778
Women Shoes                2.210948
Fashion Accessories        2.020697
Muslim Fashion             2.010608
Computer & Accessories     1.869361
Games, Books & Hobbies     1.617134
Men Shoes                  0.978640
Travel & Luggage           0.776859
Gaming & Consoles          0.742267
Tickets & Vouchers         0.458332
Others                     0.208988
Name: proportion, dtype: float64


In [777]:
delivery

,order_id,order_time,product_id,customer_id,category_1
0,order_1,2019-11-28 23:57:44.878827,13340,299,Women Clothes
1,order_2,2019-11-09 07:37:57.759814,1377,299,"Games, Books & Hobbies"
2,order_3,2019-10-02 16:07:15.863415,6999,299,Baby & Toys
3,order_4,2019-10-12 07:38:58.963653,8340,299,Women Clothes
4,order_5,2019-10-14 15:26:09.099943,11729,299,Women Clothes
...,...,...,...,...,...
69377,order_69378,2019-02-15 02:05:21.159518,1006,127744,Men Clothes
69378,order_69379,2019-06-01 16:55:39.083033,5850,127744,Groceries & Pets
69379,order_69380,2019-04-08 09:19:15.865389,13012,127744,Baby & Toys
69380,order_69381,2019-09-29 05:37:43.973087,8922,127744,Baby & Toys


In [778]:
# Generate delivery times
is_on_time = np.random.rand(len(order_dataset)) < 0.8  # 80% on-time deliveries
delivery_times = np.where(is_on_time, 
                          np.random.normal(loc=1.34, scale=0.3, size=len(order_dataset)),  # On-time deliveries
                          np.random.uniform(2, 5, size=len(order_dataset)))  # Late deliveries between 2 to 5 days

delivery_times = np.maximum(1, delivery_times).round(1)  # Ensure no delivery times below 1 day

# Add delivery times and calculate delivered_date
delivery['delivery_time'] = delivery_times
delivery['delivered_date'] = order_dataset['order_time'] + pd.to_timedelta(delivery['delivery_time'], unit='D')

# Define the delivery costs based on the category_1 column
def assign_delivery_cost(row):
    category = row['category_1']  # Use category_1 instead of generating a new category
    
    # Define the courier costs based on the options you gave
    if category in ['Men Clothes', 'Women Clothes']:
        return 3.20  # SmartPac delivery for clothes
    elif category == 'Health & Beauty':
        return 3.30  # Ninja Packs (XS) for Health & Beauty
    elif category == 'Mobile & Accessories':
        return 3.98  # Ninja Packs (M) for Mobile & Accessories
    elif category == 'Home & Living':
        return 4.30  # Qxpress Delivery 5kg for Home items
    elif category == 'Automotive':
        return 6.90  # Qxpress Delivery 10kg for Automotive
    elif category == 'Sports & Outdoor':
        return 4.48  # Ninja Packs (Padded M) for Sports & Outdoor
    elif category == 'Groceries & Pets':
        return 3.20  # SmartPac for Groceries & Pets
    elif category == 'Home Appliances':
        return 5.30  # Qxpress Delivery 10kg for Home Appliances
    elif category in ['Women\'s Bags', 'Men\'s Bags & Wallets']:
        return 3.20  # SmartPac for Bags
    elif category in ['Watches', 'Cameras & Drones']:
        return 7.30  # GrabExpress for Watches and Cameras
    else:
        return 3.98  # Default delivery cost

# Apply the delivery cost assignment function to each row based on category_1
delivery['delivery_cost'] = delivery.apply(assign_delivery_cost, axis=1)

delivery

,order_id,order_time,product_id,customer_id,category_1,delivery_time,delivered_date,delivery_cost
0,order_1,2019-11-28 23:57:44.878827,13340,299,Women Clothes,1.5,2019-11-30 11:57:44.878827,3.20
1,order_2,2019-11-09 07:37:57.759814,1377,299,"Games, Books & Hobbies",1.6,2019-11-10 22:01:57.759814,3.98
2,order_3,2019-10-02 16:07:15.863415,6999,299,Baby & Toys,1.2,2019-10-03 20:55:15.863415,3.98
3,order_4,2019-10-12 07:38:58.963653,8340,299,Women Clothes,2.2,2019-10-14 12:26:58.963653,3.20
4,order_5,2019-10-14 15:26:09.099943,11729,299,Women Clothes,1.0,2019-10-15 15:26:09.099943,3.20
...,...,...,...,...,...,...,...,...
69377,order_69378,2019-02-15 02:05:21.159518,1006,127744,Men Clothes,1.3,2019-02-16 09:17:21.159518,3.20
69378,order_69379,2019-06-01 16:55:39.083033,5850,127744,Groceries & Pets,2.3,2019-06-04 00:07:39.083033,3.20
69379,order_69380,2019-04-08 09:19:15.865389,13012,127744,Baby & Toys,1.1,2019-04-09 11:43:15.865389,3.98
69380,order_69381,2019-09-29 05:37:43.973087,8922,127744,Baby & Toys,1.3,2019-09-30 12:49:43.973087,3.98


In [779]:
(delivery.delivery_time > 2).value_counts(normalize=True)*100

delivery_time
False    79.767087
True     20.232913
Name: proportion, dtype: float64

In [780]:
delivery.category_1.value_counts(normalize=True)*100

category_1
Health & Beauty           12.248134
Men Clothes               12.089591
Women Clothes             11.024473
Mobile & Accessories       9.271857
Baby & Toys                7.243954
Home & Living              6.644375
Groceries & Pets           5.714739
Home Appliances            4.594852
Automotive                 3.999596
Sports & Outdoor           3.783402
Women's Bags               3.121847
Watches                    2.662074
Men's Bags & Wallets       2.418495
Cameras & Drones           2.288778
Women Shoes                2.210948
Fashion Accessories        2.020697
Muslim Fashion             2.010608
Computer & Accessories     1.869361
Games, Books & Hobbies     1.617134
Men Shoes                  0.978640
Travel & Luggage           0.776859
Gaming & Consoles          0.742267
Tickets & Vouchers         0.458332
Others                     0.208988
Name: proportion, dtype: float64

In [781]:
delivery['delivery_id'] = np.arange(1, len(delivery) + 1)
delivery

,order_id,order_time,product_id,customer_id,category_1,delivery_time,delivered_date,delivery_cost,delivery_id
0,order_1,2019-11-28 23:57:44.878827,13340,299,Women Clothes,1.5,2019-11-30 11:57:44.878827,3.20,1
1,order_2,2019-11-09 07:37:57.759814,1377,299,"Games, Books & Hobbies",1.6,2019-11-10 22:01:57.759814,3.98,2
2,order_3,2019-10-02 16:07:15.863415,6999,299,Baby & Toys,1.2,2019-10-03 20:55:15.863415,3.98,3
3,order_4,2019-10-12 07:38:58.963653,8340,299,Women Clothes,2.2,2019-10-14 12:26:58.963653,3.20,4
4,order_5,2019-10-14 15:26:09.099943,11729,299,Women Clothes,1.0,2019-10-15 15:26:09.099943,3.20,5
...,...,...,...,...,...,...,...,...,...
69377,order_69378,2019-02-15 02:05:21.159518,1006,127744,Men Clothes,1.3,2019-02-16 09:17:21.159518,3.20,69378
69378,order_69379,2019-06-01 16:55:39.083033,5850,127744,Groceries & Pets,2.3,2019-06-04 00:07:39.083033,3.20,69379
69379,order_69380,2019-04-08 09:19:15.865389,13012,127744,Baby & Toys,1.1,2019-04-09 11:43:15.865389,3.98,69380
69380,order_69381,2019-09-29 05:37:43.973087,8922,127744,Baby & Toys,1.3,2019-09-30 12:49:43.973087,3.98,69381


In [782]:
SG_customer

,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,...,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,attr_1,attr_2,attr_3,age,domain,checkout_count_last_1_year
0,299,32,19,1,2,0,1,3,34,118,...,2,7,0,2393,1.0,1.0,1.0,38.0,@gmail.com,56
1,2143,32,21,2,7,0,2,4,20,68,...,3,10,0,2429,1.0,1.0,2.0,26.0,@yahoo.com,77
2,2306,37,130,1,10,0,0,0,172,522,...,12,26,0,2434,1.0,1.0,4.0,56.0,@gmail.com,123
3,3334,32,9,13,19,1,1,2,2,10,...,1,3,0,2448,1.0,1.0,2.0,30.0,@gmail.com,9
4,5751,32,62,2,5,0,0,0,22,64,...,13,18,0,2486,1.0,1.0,2.0,27.0,@yahoo.com,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,126576,63,32,20,Never checkout,0,0,1,0,0,...,0,0,0,69594,NaN,1.0,0.0,25.0,@yahoo.com,0
2334,126948,63,5,5,Never checkout,1,3,3,0,0,...,0,0,0,69599,1.0,1.0,1.0,26.0,@gmail.com,0
2335,127209,63,85,889,Never checkout,0,0,0,0,0,...,0,0,0,69607,NaN,1.0,0.0,18.0,@yahoo.com,0
2336,127441,63,28,474,270,0,1,2,0,0,...,0,0,0,69611,1.0,1.0,1.0,30.0,@yahoo.com,7


In [783]:
# Create a mapping from seller_name to unique shop_id
seller_name_to_shop_id = {name: idx for idx, name in enumerate(products['seller_name'].unique(), start=1)}

# Assign shop_id to the products dataframe
products['seller_id'] = products['seller_name'].map(seller_name_to_shop_id)

# Display the first few rows of the updated products dataframe
products

,price_ori,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id,category_1,category_2,category_3,seller_id
0,8.034430,1081 PU SLING BAG KULIT leather Casual Beg Tan...,4.9,qqfwholesale,4.017215,179.0,179.0,128.0,🎉 WELCOME To QQF WHOLESALE \r\n😍 ALL item read...,2906,Malaysia,1,Women's Bags,Shoulder Bags,Shoulder Bags,1
1,0.439091,Stat Chinese Traditional Frog Buttons Knot Che...,5.0,stationery.my,0.233559,17.0,17.0,19.0,Chinese Traditional Frog Buttons Knot Cheongsa...,31701,China,2,Home Appliances,Sewing Machines,Sewing Machines and Accessories,2
2,2.647002,N09 READY STOCK MYFOOYIN woman handbag set 2in...,4.9,fooyin88,2.647002,8100.0,8100.0,21500.0,READY STOCK\nMEASUREMENT:\nLONG 26CM X HEIGHT ...,2044,Malaysia,3,Women's Bags,Handbags,Set Handbags,3
3,2.709284,🔥Shocking Sale🔥Cotton Comfort Nursing Bra Brea...,4.9,vivi2002,2.709284,6500.0,6500.0,3400.0,PRODUCT DESCRIPTION\nMaterial:95%Cotton+5%Span...,386,Malaysia,4,Women Clothes,Maternity Wear,Innerwear,4
4,1245.336605,Canon EOS 70D 18-55mm IS STM Original+64gb+ext...,5.0,xpro.gadget,852.957480,10.0,10.0,132.0,Canon 70D\nHighlights\nEF-S 18-55mm f/3.5-5.6 ...,36,Malaysia,5,Cameras & Drones,DSLR Cameras,DSLRs,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20307,15.757447,Elegant All-match Girls Leather Flat Shoes Pea...,4.9,amour.my,9.454468,219.0,219.0,1600.0,1 cm = 0.3937 inch \r\n-----------------------...,19076,China,20308,Baby & Toys,Girls Fashion,Girl Shoes,1526
20308,9.965184,Baju Raya kids 2023(v1) | Shopee Malaysia,4.8,mazu9,9.965184,215.0,215.0,304.0,M G FASHION\n-stylist fashionable\n-comfortab...,9406,Malaysia,20309,Baby & Toys,Girls Fashion,Girl Dresses,9226
20309,1.550832,Kain como crepe heavy premium (harga untk 0.5m...,4.7,nurlainafashionzone,0.775416,152.0,152.0,98.0,Harga utk 0.5m@setengah meter / HARGA BUKAN U...,446287,Malaysia,20310,Muslim Fashion,Kain Pasang & Songket,Kain Pasang Cotton,3749
20310,0.093424,Doorgift Bag Birthday Kids Drawstring Organize...,4.9,pbs_empire,0.093424,16.0,16.0,27.0,Custom made Ready Stock in Malaysia\nUnicorn S...,3695,Malaysia,20311,Baby & Toys,Others,Others,9227
